<a href="https://colab.research.google.com/github/atishaye/TigerGraph-Vid-Rec/blob/main/TigerGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly --upgrade

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
!pip install pyTigerGraph

In [ ]:
import pyTigerGraph as tg 

TG_SUBDOMAIN = 'tiger-graph' 
TG_HOST = "https://" + TG_SUBDOMAIN + ".i.tgcloud.io" # GraphStudio Link

TG_USERNAME = "tigergraph" # This should remain the same... 
TG_PASSWORD = "tigergraph" # Shh, it's our password! 
TG_GRAPHNAME = "TikTok" # The name of the graph 

conn = tg.TigerGraphConnection(host=TG_HOST, graphname=TG_GRAPHNAME, username=TG_USERNAME, password=TG_PASSWORD)
conn.apiToken = conn.getToken(conn.createSecret())

conn.gsql('ls')

In [ ]:
# Install the necessary libraries

!pip install -q jupyter-dash
!pip install dash_bootstrap_components

# Import Pandas, Datetime, and Plotly Express

import pandas as pd
from datetime import datetime
import plotly.express as px

# Import Dash Libraries

from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table
import dash

import plotly.express as px
import plotly.graph_objects as go

In [3]:
names = ["leo", "ben", "matt", "don", "joe"]
per = 25
num_coun = {}

for i in names:
    num_coun[i] = {}
    query = conn.runInstalledQuery("recommendVideo", params={"targetUsername": i, "minWatchPercent": per})
    score = 0
    for j in query[0]["output"]:
        if j['attributes']['@score'] != -1:
            score += 1
            country = j['attributes']['country']
            if country not in num_coun[i]:
                num_coun[i][country] = 1
            else:
                num_coun[i][country] += 1

print(num_coun)

{'leo': {'Canada': 1, 'USA': 2, 'China': 1, 'Japan': 1}, 'ben': {'USA': 1, 'Mexico': 1, 'Canada': 2, 'China': 1}, 'matt': {'Mexico': 1, 'China': 1, 'USA': 3, 'Canada': 1, 'Japan': 1}, 'don': {'Canada': 1, 'USA': 3, 'Japan': 1}, 'joe': {'USA': 2, 'Canada': 2, 'Mexico': 1, 'China': 1}}


In [4]:
match_TnA = {}
for i in names:
    query = conn.runInstalledQuery("recommendVideo", params={"targetUsername": i, "minWatchPercent": 25})
    ctr = 0
    for j in query[0]['output']:
        if j['attributes']['@score'] != -1:
            ctr += j['attributes']['@matchingTagsAndAudios']
    match_TnA[i] = ctr

In [5]:
scatter_per = {}
for i in names:
    scatter_per[i] = {}
    for per in range(0, 101, 25):
        query = conn.runInstalledQuery("recommendVideo", params={"targetUsername": i, "minWatchPercent": 25})
        scatter_per[i][per] = query[0]["output"][0]['attributes']['@score']

In [6]:
countries = set()
for i in num_coun:
    for j in num_coun[i]:
        countries.add(j)

for i in countries:
    for j in num_coun:
        if i not in num_coun[j]:
            num_coun[j][i] = 0

In [9]:
titleCard =  dbc.Card([
                dbc.CardBody([
                              html.H1("Analytical Video Recommendation System", className='card-title'),
                            ])
              ],
              color='dark',
              inverse=True,   
              style={
                  "width":"55rem",
                  "margin-left":"1rem",
                  "margin-top":"1rem",
                  "margin-bottom":"1rem"
                  }
            )

In [10]:
def getBar():
    df = pd.DataFrame.from_dict(num_coun, orient='index')
    fig = px.bar(df, x=list(countries), y=names, barmode = 'relative')
    fig.update_layout(
        title="Video Origin", # Adding a title
        yaxis_title="Name", # Changing x-axis label
        xaxis_title="No. of videoes", # Changing y-axis label
        yaxis_tickangle=45,
        font=dict(
            family="Courier New, monospace", # The font style
            color="RebeccaPurple", # The font color
            size=18 # The font size
        )
    )
    return fig

CountBarCard =  dbc.Card([
                dbc.CardBody([
                            html.H1("Geo Based", className='card-title'),
                            html.P("No. of recommended videos based on their coutry of origin. (Hover over individual colors to see the numerical values)", className='card-body'),  
                            dcc.Graph(id='Bar Chart', figure=getBar())
                            ])
            ],
            outline=True,
            color='info', # Options include: primary, secondary, info, success, warning, danger, light, dark  
            style={
                "width":"50rem",
                "margin-left":"1rem",
                "margin-bottom":"1rem"
                }
            )

In [11]:
def getPie():
    df = pd.DataFrame.from_dict(match_TnA, orient='index')
    fig = px.pie(df, values=list(match_TnA.values()), names = names, title= "Matching Tags", hole=0.2)
    fig.update_traces(textposition='inside',
                  textinfo='label+percent',
                  opacity=0.9
                  )
    fig.update_layout(width=1000)
    fig.update_layout(title_x=0.5)
    return fig

CountPieCard =  dbc.Card([
                dbc.CardBody([
                            html.H1("Tag Based", className='card-title'),
                            html.P("Relation between tags of user watched videos and recommended videos. (Hover over individual colors to see the numerical values)", className='card-body'),  
                            dcc.Graph(id='Pie Chart', figure=getPie())
                            ])
            ],
            outline=True,
                  color='info',
                  style={
                      "width":"30rem",
                      "margin-left":"1rem",
                      "margin-bottom":"1rem"
                      
                      }
                )

In [12]:
#Main
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

namesList = [{'label':x, 'value':x} for x in names]
jump_per = [i for i in range(0, 101, 25)]

app.layout = html.Div(children=[
    dbc.Row([
            titleCard,
           ],
           justify = 'center',
           style = {
               "margin-left": "0.5rem"
           }
   ),

    dbc.Row([
        dcc.Dropdown(
            id="example-input",
            placeholder="Enter a name..." ,
            options=namesList
        ),
        ], style={'width': '15%', 'display': 'inline-block'}
    ),
    dbc.Row([
        html.Table([
            html.Tr(children=[html.Td(["Name"]), html.Td(["Watch %"]), html.Td(["Top Recommendation (Video Id)"])],
                    style={"text-align":"center", "background-color":"#BB8FCE", "color":'black'}),
            html.Tr(children=[html.Td(id = 'my-output-name'), html.Td(id = 'my-output-per'), html.Td(id = 'my-output-id')],
                    style={"marginTop": "20px", "marginBottom": "20px", "text-align":"center",
                    "background-color":"#F5B7B1", "color":'#515A5A', "width":"100%",
                    "padding-top": "10px", "padding-bottom": "10px"})
        ])
    ]),
    
    dbc.Row([
        dcc.Slider(
            id='my-slider',
            min=0,
            max=100,
            step=25, 
            value = 10,
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ]), 
    dbc.Row([
            dbc.Col([
                     CountBarCard
                    ]),
            dbc.Col([
                     CountPieCard
                    ])
           ],
           justify = "around",
           style = {
               "margin-left": "0.5rem",
               "margin-right":"0.5rem"
           }
    ),
    dbc.Row([
        dcc.Graph(id='graph-with-button'),
        
    ]),
    dbc.Row([
        html.P("     Graph depicting how the confidence score of the top reommended video changes with the minimum watch percentage to count as a view per user.")
    ]),
    dbc.Row([
        dcc.RadioItems(id='name-button',
                        options=[
                                {'label':name, 'value':name} for name in names
                        ],
                        value='leo',
                        labelStyle={'display': 'inline-block'},
                       inputStyle={"margin-left": "50px"},
        )
    ])
])

@app.callback(
    Output('my-output-name', 'children'),
    Output('my-output-per', 'children'),
    Output('my-output-id', 'children'),
    Input('my-slider', 'value'),
    Input('example-input', 'value')
)

def update_output(per, name):
    if name is None:
        return ("N/A", "N/A", "N/A")
    q = conn.runInstalledQuery("recommendVideo", params={"targetUsername": name, "minWatchPercent": per})
    #print(q)
    video = q[0]['output'][0]['v_id']
    new_name = name.title()
    return new_name, per, video

@app.callback(
    Output('graph-with-button', 'figure'),
    Input('name-button', 'value'))
def update_figure(selected_name):
    df = pd.DataFrame.from_dict(scatter_per[selected_name], orient='index')

    fig = px.scatter(df, x=jump_per, y=list(scatter_per[selected_name].values()), size=0,
                 hover_name=0, color = 0, log_x=True, size_max=60)

    fig.update_layout(transition_duration=300, title = "Watch% vs Top Video score",
                      xaxis_title = "Watch %", yaxis_title = "Top Score")

    return fig

app.run_server(mode = 'external')

Dash app running on:


<IPython.core.display.Javascript object>